Building a Neural Network

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline


2025-04-15 17:15:14.773519: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 17:15:14.885518: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 17:15:14.976431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744717515.062843  131162 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744717515.088188  131162 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744717515.273527  131162 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [9]:
df = pd.read_csv("insurance_data.csv")
df.head()


,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [11]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100


In [12]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5


/home/zero/VSC/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-15 17:19:08.919937: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - accuracy: 0.5000 - loss: 0.7113
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5000 - loss: 0.7110
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5000 - loss: 0.7106
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5000 - loss: 0.7102
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5000 - loss: 0.7098


In [13]:
model.evaluate(X_test_scaled,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.5000 - loss: 0.7305


[0.730549156665802, 0.5]

In [14]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


array([[0.8111734 ],
       [0.7629826 ],
       [0.5630873 ],
       [0.78050345],
       [0.8142026 ],
       [0.8315921 ]], dtype=float32)

In [17]:
coef, intercept = model.get_weights()
coef, intercept

(array([[0.99500155],
        [0.9950011 ]], dtype=float32),
 array([-0.00499922], dtype=float32))

In [16]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))


In [2]:
sigmoid(18)

0.9999999847700205

In [4]:
def prediction_function(age, affordability):
    weighted_sum = coef[0]*age + coef[1]*affordability + intercept
    return sigmoid(weighted_sum)

In [18]:
prediction_function(.47,1)

/tmp/ipykernel_131162/1427957516.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1+math.exp(-x))


0.8111733787188248

In [19]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new=  [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))


In [21]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [32]:
def gradient_descent(age, affordability, y_true, epochs,loss_threshold):
    w1=w2=1
    bias = 0
    rate = 0.5
    n = len(age)

    for i in range(epochs):
        weighted_sum = w1*age + w2*affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)

        loss= log_loss(y_true, y_predicted)
        w1d=(1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d=(1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1-rate*w1d
        w2 = w2-rate*w2d
        bias = bias - bias_d
        print(f'Epoch: {i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_threshold:
            break
    return w1,w2,bias


In [33]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train,1000,0.4631)

Epoch: 0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.22683735472737165, loss:0.7113403233723417
Epoch: 1, w1:0.9601338379348763, w2:0.9123242909942049, bias:-0.40143023180012394, loss:0.6601478418560996
Epoch: 2, w1:0.9539108518006848, w2:0.8896547828871978, bias:-0.5330581497572017, loss:0.6309413150873404
Epoch: 3, w1:0.9542870421600596, w2:0.8774625582755058, bias:-0.6316685914650907, loss:0.6149458455446664
Epoch: 4, w1:0.9595254934196412, w2:0.873028852703776, bias:-0.7059780963547506, loss:0.6062106818253998
Epoch: 5, w1:0.9682458910970276, w2:0.8741231412726673, bias:-0.7628490817209272, loss:0.6012476414410665
Epoch: 6, w1:0.9794172904772812, w2:0.8790489278474481, bias:-0.8073767692539512, loss:0.5981840329519702
Epoch: 7, w1:0.9922957352752357, w2:0.8865692097444958, bias:-0.843224136943814, loss:0.5960645776348898
Epoch: 8, w1:1.0063546357556648, w2:0.8958037327693064, bias:-0.8729778688695906, loss:0.5944158714545085
Epoch: 9, w1:1.0212253267478164, w2:0.9061352839

(np.float64(5.034305567319207),
 np.float64(1.5133237783944764),
 np.float64(-3.008235220699235))

In [34]:
coef, intercept

(array([[0.99500155],
        [0.9950011 ]], dtype=float32),
 array([-0.00499922], dtype=float32))